In [1]:
# %load_ext autoreload
# %autoreload 1

In [ ]:
from robotodo.engines._isaac_legacy._kernel import Kernel
# from robotodo.engines.isaac import Scene, Camera

In [3]:
# kernel = KernelThread(["--/app/asyncRendering=True"])
kernel = Kernel([
    # "--no-window", 
    # "--/app/window/hideUi=true",
    # "--/app/asyncRendering=true",
    # TODO
    # "--enable", "omni.services.streamclient.webrtc",
])

[Warning] [simulation_app] Interactive python shell detected but ISAAC_JUPYTER_KERNEL was not set. Problems with asyncio may occur
[Warning] [simulation_app] Please use Isaac Sim Python 3 kernel instead of the default Python 3 Kernel
Loading user config located at: '/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Full/5.0/user.config.json'
2025-09-10T06:56:58Z s] [Warning] [omni.kit.app.plugin] No crash reporter present, dumps uploading isn't available.
[0.062s] [ext: omni.kit.async_engine-0.0.3] startup
[0.316s] [ext: omni.metrics.core-0.0.3] startup
[0.317s] [ext: omni.client.lib-1.1.0] startup
[0.330s] [ext: omni.blobkey-1.1.2] startup
[0.330s] [ext: omni.stats-1.0.1] startup
[0.331s] [ext: omni.datastore-0.0.0] startup
[0.339s] [ext: omni.client-1.3.0] startup
[0.395s] [ext: omni.ujitso.default-1.0.0] startup
[0.400s] [ext: omni.hsscclient-1.1.2] startup
[0.403s] [ext: omni.gpu_foundation.shadercache.vulkan-1.0.0] startup
[0.405s] [ext: omni.

Checking to see if RMW can be loaded:
failed to get symbol 'rmw_init_options_init' due to Environment variable 'AMENT_PREFIX_PATH' is not set or empty, at /workspace/humble_ws/src/rmw_implementation/src/functions.cpp:171, at /workspace/humble_ws/src/rcl/rcl/src/rcl/init_options.c:75
RMW was not loaded



In [4]:
kernel.run_update_loop()

<Future at 0x7fa85f51e8d0 state=pending>

In [5]:
# kernel.stop_update_loop()

In [6]:
from typing import TypedDict

import scipy.spatial.transform


# TODO
class ProtoPose(TypedDict):
    pass


# TODO testing
class Pose(dict):
    p: ... # TODO xyz
    q: ... # TODO xyzw
    # angles: ... # TODO radian xyz

    def __init__(
        self,
        # TODO
        # other: ProtoPose, 
        *,
        p: ... = [0, 0, 0], 
        q: ... = [0, 0, 0, 1], 
        # angles: ... = [0, 0, 0],
    ):
        super().__init__(p=p, q=q)

    # TODO 
    def facing(self, pose: "Pose"):
        # TODO
        raise NotImplementedError

    # TODO FIXME performance
    def inv(self):
        rot_inv = scipy.spatial.transform.Rotation.from_quat(self["q"]).inv()
        return Pose(
            p=-rot_inv.apply(self["p"]),
            q=rot_inv.as_quat(),
        )

    # TODO FIXME performance
    def __mul__(self, other: "Pose"):
        rot = scipy.spatial.transform.Rotation.from_quat(self["q"])
        rot_other = scipy.spatial.transform.Rotation.from_quat(other["q"])

        return Pose(
            p=self["p"] + rot.apply(other["p"]),
            q=(rot * rot_other).as_quat(),
        )


In [7]:
Pose().inv()

{'p': array([-0., -0., -0.]), 'q': array([-0., -0., -0.,  1.])}

In [8]:
Pose(p=[1, 0, 0], q=[0, 0, 0, 1]) * Pose(p=[0, 0, 0], q=[0, 0, 0, 1])

{'p': array([1., 0., 0.]), 'q': array([0., 0., 0., 1.])}

In [9]:
Pose(p=[0, 0, 0], q=[0, 0, 0, 1]).inv()

{'p': array([-0., -0., -0.]), 'q': array([-0., -0., -0.,  1.])}

In [10]:
# from robotodo.engines.core.path import Path

from typing import Iterable

# TODO
import bracex
import wcmatch.glob



class PathExpression:
    # TODO expr: "str | PathExpression | list[str | PathExpression]"
    def __init__(self, expr: "str | PathExpression"):
        self._expr = str(expr)

    def __str__(self):
        return self._expr

    # TODO this isnt correct!!!
    def resolve(self, paths: Iterable[str]):
        # TODO cache?
        # TODO brace expansion?
        match = wcmatch.glob.compile(self._expr, limit=0, flags=wcmatch.glob.GLOBSTAR)
        return match.filter(paths)

    # TODO
    def expand(self):
        return bracex.expand(self._expr)

# TODO
PathExpressionLike = str | PathExpression


import functools

# TODO multiple scenes are not supported!!!
class Scene:
    # TODO !!!!!
    def __init__(self, kernel: Kernel):
        # TODO !!!!!
        self._kernel = kernel

    # TODO !!!!!
    @property
    def _stage(self):
        # TODO !!!
        stage = kernel.omni.usd.get_context().get_stage()
        # TODO check None
        assert stage is not None
        return stage

    def traverse(self, path: PathExpressionLike | None = None):
        # TODO
        if path is not None:
            raise NotImplementedError("TODO")

        root_prim = (
            self._stage.GetPseudoRoot()
            if path is None else
            # TODO
            self._stage.GetPrimAtPath(self.resolve(path))
        )
        
        return (
            prim.GetPath().pathString
            for prim in self._kernel.pxr.Usd.PrimRange(root_prim)
        )

    def resolve(self, path: PathExpressionLike):
        return (
            PathExpression(path)
            .resolve(self.traverse())
        )
    
    def copy(self, path: PathExpressionLike, target_path: PathExpressionLike):
        path = PathExpression(path)
        target_path = PathExpression(target_path)

        def f():
            # TODO mv
            import isaacsim.core.cloner
            # TODO check if dir
            cloner = isaacsim.core.cloner.Cloner(stage=self._stage)
            source_prim_path = self.resolve(path)
            if len(source_prim_path) != 1:
                raise NotImplementedError("TODO")
            source_prim_path = source_prim_path[0]
            
            return cloner.clone(
                source_prim_path=source_prim_path,
                prim_paths=PathExpression(target_path).expand(),
            )
        # TODO
        self._kernel.submit(f).result()

        # raise NotImplementedError

    def move(self, path: PathExpressionLike, target_path: PathExpressionLike):
        path = PathExpression(path)
        target_path = PathExpression(target_path)

        raise NotImplementedError

    # TODO
    def update(self):
        raise NotImplementedError
    
    # TODO kernel thread
    @functools.cached_property
    def __physics_tensor_view_cache(self):
        try:
            # TODO stage_id !!!!!!
            res = self._kernel.omni.physics.tensors.create_simulation_view("torch")
        except Exception as error:
            # TODO
            raise RuntimeError(
                "Failed to create physics view. "
                f"Does a prim of type `PhysicsScene` exist on the stage?"
                # TODO rm
                # f"Make sure the physics simulation is running: call {self.timeline_play}"
            ) from error
        if not res.is_valid:
            raise RuntimeError(f"Created physics view is invalid: {res}")
        return res

    def _physics_tensor_get_view(self):
        if not self.__physics_tensor_view_cache.is_valid:
            del self.__physics_tensor_view_cache
        return self.__physics_tensor_view_cache
    

class Entity:

    # TODO
    @property
    def pose(self):
        raise NotImplementedError


import enum
import functools

# TODO
import torch


# TODO use kernel thread for everything
# TODO NOTE must be homogenous
class Articulation:
    def __init__(self, root_path: PathExpressionLike, scene: Scene):
        self._scene = scene
        # TODO
        self._root_path = PathExpression(root_path)

    @functools.cached_property
    def __physics_tensor_articulation_view_cache(self):
        try:
            resolved_root_paths = self._scene.resolve(self._root_path)
            self._scene._kernel.omni.physx.get_physx_simulation_interface().flush_changes()
            res = (
                self._scene._physics_tensor_get_view()
                .create_articulation_view(resolved_root_paths)
            )
            assert res is not None
            assert res.check()
        except Exception as error:
            raise RuntimeError(
                f"Failed to create physics view from root joint paths (are they valid?): {resolved_root_paths}"
            ) from error
        return res
    
    def _physics_tensor_get_articulation_view(self):
        if not self.__physics_tensor_articulation_view_cache.check():
            del self.__physics_tensor_articulation_view_cache
        return self.__physics_tensor_articulation_view_cache



    # TODO
    @property
    def root_joint_path(self):
        # TODO
        return self._physics_tensor_get_articulation_view().prim_paths
    # TODO

    @property
    def link_paths(self):
        return self._physics_tensor_get_articulation_view().link_paths
    


    @property
    def dof_count(self):
        return self._physics_tensor_get_articulation_view().max_dofs

    # TODO mv and standardize
    class DOFType(enum.Enum):
        Rotation = 0
        Translation = 1

    # TODO
    @property
    def dof_types(self):
        def f():
            view = self._physics_tensor_get_articulation_view()
            return view.get_dof_types()
        return self._scene._kernel.submit(f).result()

    @property
    def dof_positions(self):
        def f():
            view = self._physics_tensor_get_articulation_view()
            dof_positions = view.get_dof_positions()
            return dof_positions
        return self._scene._kernel.submit(f).result()
    
    # TODO maybe async???
    # TODO typing
    # TODO support masking !!!!!!
    @dof_positions.setter
    def dof_positions(self, value):
        def f():
            view = self._physics_tensor_get_articulation_view()
            value_ = torch.broadcast_to(torch.asarray(value), (view.count, view.max_dofs))            
            view.set_dof_positions(value_, indices=torch.arange(view.count))
        return self._scene._kernel.submit(f).result()

    @property
    def dof_position_limits(self):
        def f():
            view = self._physics_tensor_get_articulation_view()
            # TODO
            return view.get_dof_limits()
        return self._scene._kernel.submit(f).result()
    
    @property
    def dof_velocities(self):
        def f():
            view = self._physics_tensor_get_articulation_view()
            velocities = view.get_dof_velocities()
            return velocities
        return self._scene._kernel.submit(f).result()
    
    @dof_velocities.setter
    def dof_velocities(self, value):
        def f():
            view = self._physics_tensor_get_articulation_view()
            value_ = torch.broadcast_to(torch.asarray(value), (view.count, view.max_dofs))            
            view.set_dof_velocities(value_, indices=torch.arange(view.count))
        return self._scene._kernel.submit(f).result()
    
    @functools.cached_property
    def driver(self):
        return ArticulationDriver(articulation=self)
    
    @property
    def root_pose(self):
        def f():
            view = self._physics_tensor_get_articulation_view()
            root_trans = view.get_root_transforms()
            return Pose(
                p=root_trans[..., [0, 1, 2]],
                q=root_trans[..., [3, 4, 5, 6]],
            )
        return self._scene._kernel.submit(f).result()



class ArticulationDriver:

    def __init__(self, articulation: Articulation):
        self._articulation = articulation

    def compute_dof_passive_gravity_forces(self):
        def f():
            view = self._articulation._physics_tensor_get_articulation_view()
            # TODO extra +6 dofs when floating base !!!!!!
            # TODO indices: [*base_6dof, *dof]
            return view.get_gravity_compensation_forces()
        return self._articulation._scene._kernel.submit(f).result()
    
    def compute_dof_passive_coriolis_and_centrifugal_forces(self):
        def f():
            view = self._articulation._physics_tensor_get_articulation_view()
            # TODO extra +6 dofs when floating base !!!!!!
            return view.get_coriolis_and_centrifugal_compensation_forces()
        return self._articulation._scene._kernel.submit(f).result()
    
    # TODO mv and stdize
    class DriveType(enum.Enum):
        Disabled = 0
        Force = 1
        Acceleration = 2

    @property
    def dof_drive_types(self):
        def f():
            view = self._articulation._physics_tensor_get_articulation_view()
            return view.get_drive_types()
        return self._articulation._scene._kernel.submit(f).result()

    @property
    def dof_target_positions(self):
        def f():
            view = self._articulation._physics_tensor_get_articulation_view()
            return view.get_dof_position_targets()
        return self._articulation._scene._kernel.submit(f).result()

    @dof_target_positions.setter
    def dof_target_positions(self, value):
        def f():
            view = self._articulation._physics_tensor_get_articulation_view()
            value_ = torch.broadcast_to(torch.asarray(value), (view.count, view.max_dofs))
            view.set_dof_position_targets(value_, indices=torch.arange(view.count))
        return self._articulation._scene._kernel.submit(f).result()

    @property
    def dof_target_velocities(self):
        def f():
            view = self._articulation._physics_tensor_get_articulation_view()
            return view.get_dof_velocity_targets()
        return self._articulation._scene._kernel.submit(f).result()

    @dof_target_velocities.setter
    def dof_target_velocities(self, value):
        def f():
            view = self._articulation._physics_tensor_get_articulation_view()
            value_ = torch.broadcast_to(torch.asarray(value), (view.count, view.max_dofs))
            view.set_dof_velocity_targets(value_, indices=torch.arange(view.count))
        return self._articulation._scene._kernel.submit(f).result()



In [11]:
# scene.copy("/World/*", "/WorldA{0..1}")

In [12]:
scene = Scene(kernel=kernel)
# scene.resolve("/**")

In [13]:
# scene.copy("/Ant", "/Ant{10..16}")
# scene.copy("/Franka", "/Franka{10..16}")

In [14]:
list(scene.traverse())

[13.001s] [ext: isaacsim.ros2.bridge-4.9.3] startup
2025-09-10T06:57:11Z [13,193ms] [Warning] [isaacsim.ros2.bridge.impl.extension] ROS_DISTRO env var not found, Please source ROS before enabling this extension
[13.215s] Using backup internal ROS2 humble distro
[13.232s] To use the internal libraries included with the extension please set the following environment variables to use with FastDDS (default) or CycloneDDS before starting Isaac Sim:

FastDDS (default):
export ROS_DISTRO=humble
export RMW_IMPLEMENTATION=rmw_fastrtps_cpp
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/isaacsim/exts/isaacsim.ros2.bridge/humble/lib

OR

CycloneDDS:
export ROS_DISTRO=humble
export RMW_IMPLEMENTATION=rmw_cyclonedds_cpp
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/isaacsim/exts/isaacsim.ros2.bridge/humble/lib


[13.245s] [ext: isaacsim.ros2.bridge-4.9.3] shutdown


2025-09-10T06:57:11Z [13,210ms] [Error] [isaacsim.ros2.bridge.impl.extension] ROS2 Bridge startup failed


2025-09-10T06:57:11Z [13,434ms] [Warning] [omni.ext._impl._internal] isaacsim.ros2.bridge-4.9.3 -> <class 'isaacsim.ros2.bridge.impl.samples.ros_moveit_sample.Extension'>: extension object is still alive, something holds a reference on it. References: ["[0]:type: <class 'method'>, id: 140361364818688", "[1]:type: <class 'method'>, id: 140361364816000"]
2025-09-10T06:57:11Z [13,641ms] [Warning] [omni.ext._impl._internal] isaacsim.ros2.bridge-4.9.3 -> <class 'isaacsim.ros2.bridge.impl.samples.ros_waypoint_follower_sample.Extension'>: extension object is still alive, something holds a reference on it. References: ["[0]:type: <class 'method'>, id: 140361090820224", "[1]:type: <class 'method'>, id: 140361019882240"]
2025-09-10T06:57:11Z [13,679ms] [Warning] [omni.ext._impl._internal] isaacsim.ros2.bridge-4.9.3 -> <class 'isaacsim.ros2.bridge.impl.samples.ros_samples.Extension'>: extension object is still alive, something holds a reference on it. References: ["[0]:type: <class 'method'>, id:

AssertionError: 

2025-09-10T06:57:12Z [14,546ms] [Warning] [omni.fabric.plugin] Warning: attribute viewportHandle not found for bucket id b

2025-09-10T06:57:12Z [14,650ms] [Warning] [carb] Acquiring non optional plugin interface which is not listed as dependency: [carb::dictionary::ISerializer v1.1] (plugin: carb.dictionary.serializer-json.plugin), by client: carb.scenerenderer-rtx.plugin. Add it to CARB_PLUGIN_IMPL_DEPS() macro of a client.
[14.684s] app ready
2025-09-10T06:57:12Z [14,704ms] [Warning] [carb.graphics-vulkan.plugin] Trying to initialize NGX without vkGetInstanceProcAddr and vkGetDeviceProcAddr pointer-to-functions.
2025-09-10T06:57:12Z [14,704ms] [Warning] [carb.scenerenderer-rtx.plugin] Failed to create NGX context.
2025-09-10T06:57:12Z [14,796ms] [Warning] [rtx.optixdenoising.plugin] Failed to create an Optix 7.5 adaptor on device 0. Temporal denoising will not be available.
2025-09-10T06:57:12Z [14,796ms] [Warning] [rtx.optixdenoising.plugin] Failed to initialize OptixDenoiserContex

2025-09-10T06:57:12Z [14,704ms] [Error] [carb.graphics-vulkan.plugin] Could not get NGX parameters block because NGX isn't enabled.
2025-09-10T06:57:12Z [14,704ms] [Error] [carb.graphics-vulkan.plugin] Failed to create NGX context.
2025-09-10T06:57:12Z [14,796ms] [Error] [rtx.optixdenoising73.plugin] Failed to create an Optix adaptor for device 0


2025-09-10T06:57:14Z [16,918ms] [Warning] [usdrt.hydra.fabric_scene_delegate.plugin] using high frequency span is disabled
2025-09-10T06:57:14Z [16,918ms] [Warning] [usdrt.hydra.fabric_scene_delegate.plugin] using high frequency span is disabled
2025-09-10T06:57:14Z [16,918ms] [Warning] [usdrt.hydra.fabric_scene_delegate.plugin] using high frequency span with attrs is disabled
2025-09-10T06:57:14Z [16,946ms] [Warning] [usdrt.hydra.fabric_scene_delegate.plugin] using high frequency span is disabled
2025-09-10T06:57:14Z [16,946ms] [Warning] [usdrt.hydra.fabric_scene_delegate.plugin] using high frequency span is disabled
[17.111s] Isaac Sim Full App is loaded.
2025-09-10T06:57:20Z [22,711ms] [Warning] [usdrt.hydra.fabric_scene_delegate.plugin] using high frequency span with attrs is disabled


In [ ]:
import isaacsim.asset.importer.urdf

In [15]:

# TODO
def import_urdf(
    resource: str, 
    prim_path: str | None = None,
    fix_base: bool = True,
):
    import omni

    # TODO
    ext_manager = omni.kit.app.get_app().get_extension_manager()
    if not ext_manager.is_extension_enabled("isaacsim.asset.importer.urdf"):
        ext_manager.set_extension_enabled_immediate("isaacsim.asset.importer.urdf", True)

    # TODO
    is_success, import_config = omni.kit.commands.execute(
        "URDFCreateImportConfig"
    )
    assert is_success

    # import_config.convex_decomp = False  # Disable convex decomposition for simplicity
    import_config.fix_base = fix_base       # Fix the base of the robot to the ground
    import_config.make_default_prim = False  # Make the robot the default prim in the scene
    # import_config.self_collision = False  # Disable self-collision for performance
    # import_config.distance_scale = 1     # Set distance scale for the robot
    # import_config.density = 0.0          # Set density to 0 (use default values)

    # from isaacsim.asset.importer.urdf import _urdf

    # Acquire the URDF extension interface for parsing and importing URDF files
    # urdf_interface = _urdf.acquire_urdf_interface()

    # Configure the settings for importing the URDF file
    # import_config = _urdf.ImportConfig()

    # Retrieve the path of the URDF file from the extension


    # from isaacsim.core.utils.extensions import get_extension_path_from_name
    # extension_path = get_extension_path_from_name("isaacsim.asset.importer.urdf")
    # root_path = extension_path + "/data/urdf/robots/franka_description/robots"
    # file_name = "panda_arm_hand.urdf"
    # Parse the robot's URDF file to generate a robot model
    # urdf_path="{}/{}".format(root_path, file_name),

    is_success, robot_model = omni.kit.commands.execute(
        "URDFParseFile",
        urdf_path=resource,
        import_config=import_config
    )
    assert is_success


    # TODO  check if path already exists
    stage = omni.usd.get_context().get_stage()
    if stage.GetPrimAtPath(prim_path).IsValid():
        raise RuntimeError(f"Path already exists: {prim_path}")


    # Update the joint drive parameters for better stiffness and damping
    # for joint in robot_model.joints:
    #     robot_model.joints[joint].drive.strength = 1047.19751  # High stiffness value
    #     robot_model.joints[joint].drive.damping = 52.35988    # Moderate damping value

    # Import the robot onto the current stage and retrieve its prim path
    is_success, prim_path_temp = omni.kit.commands.execute(
        "URDFImportRobot",
        urdf_robot=robot_model,
        import_config=import_config,
        get_articulation_root=True,
    )
    assert is_success
    if prim_path is None:
        return prim_path_temp

    is_success, _ = omni.kit.commands.execute(
        "MovePrim",
        path_from=prim_path_temp,
        path_to=prim_path,
    )
    assert is_success

    return prim_path

In [16]:
import os

from isaacsim.core.utils.extensions import get_extension_path_from_name


panda_arm_hand_urdf_path = os.path.join(
    get_extension_path_from_name("isaacsim.asset.importer.urdf"),
    "data", "urdf", "robots", "franka_description", "robots", "panda_arm_hand.urdf",
)


In [17]:


panda_robot_prim_path = kernel.submit(
    lambda: import_urdf(
        panda_arm_hand_urdf_path, 
        prim_path="/World/PandaFixedBase0",
        fix_base=True,
    )
).result()


2025-09-10T06:57:55Z [57,017ms] [Warning] [isaacsim.asset.importer.urdf] link panda_hand has no body properties (mass, inertia, or collisions) and is being merged into panda_link8
2025-09-10T06:57:55Z [57,017ms] [Warning] [isaacsim.asset.importer.urdf] link panda_link8 has no body properties (mass, inertia, or collisions) and is being merged into panda_link7
2025-09-10T06:57:55Z [57,017ms] [Warning] [isaacsim.asset.importer.urdf] Creating Asset in an in-memory stage, will not create layered structure
2025-09-10T06:57:55Z [57,023ms] [Warning] [omni.usd] Warning: in SdfPath at line 144 of /builds/omniverse/usd-ci/USD/pxr/usd/sdf/path.cpp -- Ill-formed SdfPath <>: :1:0(0): parse error matching pxrInternal_v0_24__pxrReserved__::Sdf_PathParser::Path

2025-09-10T06:57:55Z [57,649ms] [Warning] [isaacsim.asset.importer.urdf] Joint panda_finger_joint2 has a velocity limit defined but is set to mimic joint panda_finger_joint1. Velocity limit ignored.


2025-09-10T06:57:55Z [57,738ms] [Warning] [usdrt.hydra.fabric_scene_delegate.plugin] using high frequency span with attrs is disabled
2025-09-10T06:57:55Z [57,739ms] [Warning] [omni.fabric.plugin] getAttributeCount called on non-existent path /World/panda/panda_rightfinger/visuals/finger
2025-09-10T06:57:55Z [57,739ms] [Warning] [omni.fabric.plugin] getTypes called on non-existent path /World/panda/panda_rightfinger/visuals/finger


In [ ]:
panda_arm_hand_urdf_path

'/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/isaacsim/exts/isaacsim.asset.importer.urdf/data/urdf/robots/franka_description/robots/panda_arm_hand.urdf'

In [ ]:
panda_robot_prim_path

'/World/PandaFixedBase0'

In [25]:
panda_robot = Articulation(panda_robot_prim_path, scene=scene)


def todo():
    return panda_robot.link_paths
kernel.submit(todo).result()

[['/World/panda/panda_link0',
  '/World/panda/panda_link1',
  '/World/panda/panda_link2',
  '/World/panda/panda_link3',
  '/World/panda/panda_link4',
  '/World/panda/panda_link5',
  '/World/panda/panda_link6',
  '/World/panda/panda_link7',
  '/World/panda/panda_leftfinger',
  '/World/panda/panda_rightfinger']]

In [26]:
panda_robot.dof_count

9

In [ ]:
from isaacsim.core.utils.prims import (
    get_articulation_root_api_prim_path,
    get_prim_at_path,
    get_prim_parent,
    get_prim_property,
    set_prim_property,
)


# get_articulation_root_api_prim_path("/World/panda")

In [ ]:
panda_robot

In [19]:
from curobo.types.robot import RobotConfig
from curobo.geom.types import WorldConfig, Mesh

from curobo.types.math import Pose as CuroboPose
from curobo.types.robot import JointState
from curobo.wrap.reacher.motion_gen import (
    MotionGen,
    MotionGenConfig,
    MotionGenPlanConfig,
    PoseCostMetric,
)


kinematics_fused_cu not found, JIT compiling...
/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/torch/utils/cpp_extension.py:2356: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
geom_cu binary not found, jit compiling...
/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/torch/utils/cpp_extension.py:2356: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
tensor_step_cu not found, jit compiling...
/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/torch/utils/cpp_extension.py:2356: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warni

In [56]:
kernel.stop_update_loop()

<Future at 0x7f9aa0808ad0 state=pending>

In [57]:
# TODO
motion_gen = MotionGen(
    MotionGenConfig.load_from_robot_config(
        robot_cfg=RobotConfig.from_basic(
            # TODO '/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/isaacsim/exts/isaacsim.asset.importer.urdf/data/urdf/robots/franka_description/robots/panda_arm_hand.urdf'
            urdf_path=panda_arm_hand_urdf_path,
            base_link="panda_link0",
            ee_link="panda_link7",
        ),
        world_model=WorldConfig(
            # TODO
            # mesh=curobo_meshes,
        ),
        interpolation_dt=1 / 60,
        num_trajopt_seeds=1,
        num_graph_seeds=1,
    )
)
motion_gen.warmup()


RuntimeError: Offset increment outside graph capture encountered unexpectedly.

In [21]:
# scene = Scene(kernel=kernel)
# panda_robot = Articulation("/World/PandaFixedBase0", scene=scene)


In [22]:
scene

In [43]:
motion_gen.kinematics.joint_names

['panda_joint1',
 'panda_joint2',
 'panda_joint3',
 'panda_joint4',
 'panda_joint5',
 'panda_joint6',
 'panda_joint7']

In [32]:
panda_robot.dof_positions

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [33]:
panda_robot.dof_positions.shape

torch.Size([1, 9])

In [34]:
panda_robot.link_paths

[['/World/panda/panda_link0',
  '/World/panda/panda_link1',
  '/World/panda/panda_link2',
  '/World/panda/panda_link3',
  '/World/panda/panda_link4',
  '/World/panda/panda_link5',
  '/World/panda/panda_link6',
  '/World/panda/panda_link7',
  '/World/panda/panda_leftfinger',
  '/World/panda/panda_rightfinger']]

In [35]:
panda_robot.root_joint_path

['/World/PandaFixedBase0']

In [36]:
# from robotodo.engines.core.pose import Pose

In [37]:
panda_robot.link_paths

[['/World/panda/panda_link0',
  '/World/panda/panda_link1',
  '/World/panda/panda_link2',
  '/World/panda/panda_link3',
  '/World/panda/panda_link4',
  '/World/panda/panda_link5',
  '/World/panda/panda_link6',
  '/World/panda/panda_link7',
  '/World/panda/panda_leftfinger',
  '/World/panda/panda_rightfinger']]

In [38]:
panda_robot.dof_positions

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [53]:
dof_positions = panda_robot.dof_positions[..., :-2]
# TODO 
target_pose = panda_robot.root_pose.inv() * Pose(p=[.5, .2, .5], q=[0, 0, 0, 1])
res = motion_gen.plan_single(
    JointState.from_position(position=torch.tensor(dof_positions, dtype=torch.float32).cuda()),
    CuroboPose(position=torch.tensor(target_pose["p"], dtype=torch.float32).cuda()[0], quaternion=torch.tensor(target_pose["q"], dtype=torch.float32).cuda()[..., [1, 2, 3, 0]])[0], 
    MotionGenPlanConfig(max_attempts=100),
)
res

/tmp/ipykernel_3519734/544185721.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  JointState.from_position(position=torch.tensor(dof_positions, dtype=torch.float32).cuda()),
Couldn't find solution with 10 attempts, resetting seeds


MotionGenResult(success=tensor([False], device='cuda:0'), valid_query=True, optimized_plan=JointState(position=tensor([[[ 0.7150,  0.9624,  0.3518, -0.7608, -0.5397,  1.8478,  1.0050],
         [ 0.7150,  0.9624,  0.3518, -0.7608, -0.5397,  1.8478,  1.0050],
         [ 0.7131,  0.9570,  0.3493, -0.7693, -0.5375,  1.8518,  1.0018],
         [ 0.7094,  0.9456,  0.3442, -0.7870, -0.5328,  1.8600,  0.9952],
         [ 0.7038,  0.9287,  0.3367, -0.8133, -0.5259,  1.8722,  0.9854],
         [ 0.6965,  0.9067,  0.3268, -0.8475, -0.5169,  1.8880,  0.9727],
         [ 0.6877,  0.8800,  0.3149, -0.8889, -0.5059,  1.9070,  0.9573],
         [ 0.6775,  0.8492,  0.3012, -0.9367, -0.4933,  1.9290,  0.9395],
         [ 0.6661,  0.8147,  0.2858, -0.9902, -0.4792,  1.9536,  0.9195],
         [ 0.6535,  0.7770,  0.2689, -1.0487, -0.4637,  1.9805,  0.8977],
         [ 0.6401,  0.7364,  0.2508, -1.1116, -0.4470,  2.0093,  0.8742],
         [ 0.6259,  0.6936,  0.2316, -1.1780, -0.4294,  2.0397,  0.8493],
 

In [ ]:
# kernel.stop_update_loop()

<Future at 0x7f9fe7a978d0 state=pending>

In [32]:
kernel.run_update_loop()

<Future at 0x7f9aa0747110 state=pending>

In [31]:
res.success

tensor([False], device='cuda:0')

In [67]:
import numpy

numpy.allclose([1e-12], 0, atol=1e-6)

True

In [35]:
res.interpolated_plan.position.shape

torch.Size([1, 77, 7])

In [54]:
import numpy

for pos, vel in zip(
    res.interpolated_plan.position[0],
    res.interpolated_plan.velocity[0],
):
    dof_target_pos = panda_robot.driver.dof_target_positions
    dof_target_pos[..., :-2] = pos
    panda_robot.driver.dof_target_positions = dof_target_pos

    dof_target_vel = panda_robot.driver.dof_target_velocities
    dof_target_vel[..., :-2] = vel
    panda_robot.driver.dof_target_velocities = dof_target_vel

    pos_err = panda_robot.dof_positions - panda_robot.driver.dof_target_positions
    vel_err = panda_robot.dof_velocities - panda_robot.driver.dof_target_velocities
    
    # TODO FIXME: use events!@!!!!! upstream: await SomeRef.allclose(0, atol=...).event()
    while not all([
        numpy.allclose(pos_err, 0, atol=1e-1),
        # numpy.allclose(vel_err, 0, atol=1e-1),
    ]): 
        pass
    


In [45]:
vel_err

tensor([[ 0.0035, -0.0087,  0.0033, -0.0305,  0.0009,  0.3050,  0.0062, -0.0189,
         -0.0189]])

In [44]:
numpy.allclose(vel_err, 0, atol=1e-1)

False

In [40]:
pos_err

tensor([[ 3.1156e-05,  2.1015e-04,  2.0373e-05, -1.3709e-06, -5.7719e-05,
          3.6613e-04,  6.9316e-05,  2.7233e-04, -7.7830e-03]])

In [35]:
torch.asarray(target_pose["p"]).cuda()

tensor([1, 0, 1], device='cuda:0')

In [97]:
panda_arm_hand_urdf_path

'/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/isaacsim/exts/isaacsim.asset.importer.urdf/data/urdf/robots/franka_description/robots/panda_arm_hand.urdf'

In [41]:
todo

'/panda/root_joint'

In [ ]:
_ = """

scene = Scene()
camera = scene.create(
    "/path/to/camera",
    entity_or_ref=Camera(resolution=(1, 1)),
)
articulation = scene.create(
    "/path/to/robot",
    resource="file:/some/path/to/robot.urdf",
)
some_object = scene.create(
    "/path/to/some_object",
    spec=RigidBody(...),
)


scene.copy(
    "/path/to/robot",
    "/path/to/robot_copy",
)

scene.get(...)

scene.update()


another_scene = Scene(children={
    "World": {
        "SomeRobot": Entity(
            components=[ArticulationRoot(), RigidBody()],
            children={
                "Camera": Camera(resolution=(1, 1)),
            },
        ),
        "Camera": Camera(resolution=(1, 1)),
    },
})

another_scene.at("/World/SomeRobot")


entity = Entity(
    "/World/SomeObject",
    traits=[RigidTrait(), ColliderTrait()],
)

entity.pose = ...

articulation = Articulation()

# scene

# scene.add(Entity.from("file:/path/to.urdf"))
# TODO
# scene.add_usd("/yep")
# scene.add_urdf("/World/SomeRobot", resource="file:/path/to/some.urdf")

"""

In [ ]:
# Articulation.load_urdf(...)

In [ ]:
# Scene.load_usd
# Scene.load
# scene = Scene.load_usd("some.usd")
# scene.add_usd
# scene.add_urdf
# scene.add
# scene.save

In [37]:
kernel.stop_update_loop()

<Future at 0x7f431c27be10 state=pending>

In [ ]:
from omni.isaac.core.prims import XFormPrimView